In [1]:
from ped_car_2 import PedestrianEnv
import numpy as np
from My_DDQN import DDQN
import random
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
env=PedestrianEnv()
observation = env.reset()
observation_space=len(observation) #we get the number of parameters in the state
action_space=5 #number of discrete velocities pedestrian can take
agent=DDQN(observation_space, action_space)
episodes=1000

In [ ]:
reward_list=[0]
for e in range(episodes):
    episode_reward=0
    state=env.reset()
    state = np.reshape(state, [1, observation_space])    
    while True:
        action=agent.act(state)
        action_1=action*((env.P_velocity_limit)/(action_space-1))
        state_next, reward, terminal = env.step(np.array([action_1]))
        episode_reward+=reward
        state_next = np.reshape(state_next, [1, observation_space])
        agent.add_memory(state, action, reward, state_next, terminal)
        state = state_next
        if(len(agent.memory)>agent.minibatch_size):
            agent.replay()
        if terminal:
            print ("Episode: " + str(e) + ", exploration: " + str(agent.exploration_rate) + ", score: " + str(episode_reward))            
            agent.target_model_update()
            reward_list.append((reward_list[len(reward_list)-1]*e+episode_reward)/(e+1))
            break
    if agent.exploration_rate>agent.exploration_min:
        agent.exploration_rate*=agent.exploration_decay       
    

Episode: 0, exploration: 1, score: 4095.5
Episode: 1, exploration: 0.995, score: 4187.2
Episode: 2, exploration: 0.990025, score: 4174.4
Episode: 3, exploration: 0.985074875, score: -10067.90804957848
Episode: 4, exploration: 0.9801495006250001, score: 4187.2
Episode: 5, exploration: 0.9752487531218751, score: -10155.702053623043
Episode: 6, exploration: 0.9703725093562657, score: 3970.3999999999996
Episode: 7, exploration: 0.9655206468094844, score: 4068.4
Episode: 8, exploration: 0.960693043575437, score: 4174.4
Episode: 9, exploration: 0.9558895783575597, score: -10052.151853865547
Episode: 10, exploration: 0.9511101304657719, score: 4027.0
Episode: 11, exploration: 0.946354579813443, score: -10098.28789568814
Episode: 12, exploration: 0.9416228069143757, score: 4040.9
Episode: 13, exploration: 0.9369146928798039, score: -10086.257643967743
Episode: 14, exploration: 0.9322301194154049, score: 4095.5
Episode: 15, exploration: 0.9275689688183278, score: 3998.9
Episode: 16, exploration

In [ ]:
plt.figure(1)
plt.plot(reward_list)
plt.show()

In [ ]:
observation = env.reset()
death_toll=0
safe_chicken=0
done_count=0
agent.exploration_rate=0
for t in range(700): 
    action = agent.act(state)
    action_1=action*((env.P_velocity_limit)/(action_space-1))
    observation, reward, done = env.step(np.array([action_1]))          
    if done:       
        done_count+=1
        if (reward==-10000):
            death_toll+=1                                              
        if (reward==5000):           
            safe_chicken+=1        
        env.reset()
print('Death_toll '+str(death_toll))
print('Episodes', done_count)
print('Death_toll % '+str(death_toll*100/(done_count)))
print('Safe_chicken',safe_chicken)
print('Safe_toll % '+str(safe_chicken*100/(done_count)))

In [ ]:
N = 100
cumsum, moving_aves = [0], []

for i, x in enumerate(reward_list, 1):
    cumsum.append(cumsum[i-1] + x)
    if i>=N:
        moving_ave = (cumsum[i] - cumsum[i-N])/N
        #can do stuff with moving_ave here
        moving_aves.append(moving_ave)

plt.figure(2)
plt.plot(moving_aves)
plt.show()